Задача по каждому производителю (поле manufacturer_name):
- кол-во объявлений
- средний год выпуска автомобилий
- минимальную цену
- максимальную цену
    
Выгрузить результат в output.csv 

In [24]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [35]:
def main(): 
    spark = SparkSession.builder.getOrCreate()
    df = spark.read.format("csv").option('header', 'True').load("data/cars.csv")
    
    output = (
        df
        .groupBy("manufacturer_name")
        .agg(
            F.count("manufacturer_name").alias("count"), 
            F.round(F.avg("year_produced")).cast('int').alias("avg_year"), 
            F.min(F.col("price_usd").cast('float')).alias("min_price"), 
            F.max(F.col("price_usd").cast('float')).alias("max_price")
        )
    )

    output.show(5)
    output.coalesce(1).write.mode("overwrite").format('json').save('data/result.json')
main()

+-----------------+-----+--------+---------+---------+
|manufacturer_name|count|avg_year|min_price|max_price|
+-----------------+-----+--------+---------+---------+
|       Volkswagen| 4243|    2002|      1.0|  43999.0|
|            Lexus|  213|    2008|   2500.0| 48610.45|
|           Jaguar|   53|    2009|   2500.0|  50000.0|
|            Rover|  235|    1998|    200.0|   9900.0|
|           Lancia|   92|    2000|    200.0|   9500.0|
+-----------------+-----+--------+---------+---------+
only showing top 5 rows

